In [1]:
import xlwings as xw
import pandas as pd
from collections import defaultdict
import os
from upath import UPath

In [2]:
# Do the xlwings thing where the SharePoint path is updated
# Format this notebook before committing
# Copy Run.jl
# Copy settings
# Check that demand_data, fuel_data, and generator_variability CSVs have the same length

In [3]:
wb = xw.Book('/Users/roderick/Library/CloudStorage/OneDrive-SharedLibraries-ResilientTransition/5.001 Kentucky Resource Council - Documents/Data/Kentucky Load Resource Model.xlsb')

In [23]:
def save_case(base_folder: UPath, case_subfolder: str | None = None):
    global name, range, col
    # Get CSV names as a nested dictionary (since some CSVs have been split into multiple separate tables
    # Named ranges have the format of [csv file name]...[#]...[optional transformation, either .T or .ffill]
    csv_names = defaultdict(list)
    for name in wb.names:
        if ".csv" in name.name:
            csv_names[name.name.split("...")[0]].append(name)
    for csv_name, ranges in csv_names.items():
        dfs = []
        for range in ranges:
            # Get each range as a dataframe
            df = range.refers_to_range.options(pd.DataFrame, index=0,
                                               header=(1 if not range.name.endswith("...T") else 0)).value
            df = df.dropna(how="all", axis=1)
            df = df.dropna(how="all", axis=0)
            if "resource" in df.columns:
                df = df.dropna(subset="resource", axis=0)
            if "drop" in df.columns:
                df = df[df["drop"] != True]

            # Apply optional transform
            if range.name.endswith("...T"):
                df = df.set_index(df.columns[0])
                df = df.T
            elif range.name.endswith("...ffill"):
                df = df.ffill()
            elif range.name.endswith("...drop...1"):
                df = df.iloc[:, [0, -1]]
                df = df.dropna(how="any")
            elif range.name.endswith("...drop...3"):
                df = df.iloc[:, [0, -3, -2, -1]]
                df = df.dropna(how="any")

            if csv_name in [
                "resources\\policy_assignments\\Resource_NQC_derate.csv",
                "resources\\policy_assignments\\ELCC_multipliers.csv"
            ]:
                df = df.rename(columns={"resource": "Resource"})

            # Change types for columns to int & strings
            int_columns = [
                col for col in df.columns if col in
                                             [
                                                 "can_retire",
                                                 "zone",
                                                 "new_build",
                                                 "model",
                                                 "lds",
                                                 "Time_Index"
                                             ]
            ]
            df[int_columns] = df[int_columns].astype(int)

            str_columns = [
                col for col in df.columns if col in
                                             [
                                                 "cluster",
                                                 "region",
                                             ]
            ]
            df[str_columns] = df[str_columns].astype(str)

            dfs.append(df)

        # Join all the dfs
        final_df = pd.concat([df.reset_index(drop=True) for df in dfs], axis=1)

        # Save joined dataframe to CSV
        planning_period_folder = base_folder / case_subfolder if case_subfolder else base_folder
        filepath = planning_period_folder / csv_name.replace("\\", os.sep)
        filepath.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_csv(filepath, index=False)

In [28]:
base_folder = UPath(wb.names["BaseFolder"].refers_to_range.value)

planning_periods = wb.sheets["GenX Settings"].tables["ModeledYears"].range.options(pd.DataFrame, index=1).value.dropna().index.astype(int).values

counter = 1
for planning_period in planning_periods:
    wb.sheets["GenX Settings"].range("ActiveYear").value = planning_period
    wb.app.calculate()

    print(f"Saving case inputs for {planning_period}: (inputs_p{counter})")
    save_case(base_folder=base_folder, case_subfolder=f"inputs_p{counter}")
    counter += 1

Saving case inputs for 2025 (p1)
Saving case inputs for 2030 (p2)
Saving case inputs for 2040 (p3)
Saving case inputs for 2050 (p4)


## Running Case

In [6]:
!julia --project=. Run.jl "/Users/roderick/PycharmProjects/resilient-transition/GenX.jl/krc"

]0;Julia]0;Julia  ____           __  __   _ _
 / ___| ___ _ __ \ \/ /  (_) |
| |  _ / _ \ '_ \ \  /   | | |
| |_| |  __/ | | |/  \ _ | | |
 \____|\___|_| |_/_/\_(_)/ |_|
                       |__/
 Version: 0.4.4

Configuring Settings
Time Series Data Already Clustered.
Configuring Solver
Loading Inputs
Reading Input CSV Files
Demand (load) data Successfully Read!
Fuels_data.csv Successfully Read!
[ Info: Hydro.csv Successfully Read.
[ Info: Thermal.csv Successfully Read.
[ Info: Vre.csv Successfully Read.
[ Info: Storage.csv Successfully Read.
[ Info: Must_run.csv Successfully Read.

Summary of resources loaded into the model:
-------------------------------------------------------
	Resource type 		Number of resources
	Thermal        		40
	VRE            		14
	Hydro          		1
	Storage        		4
	Must_run       		1
Total number of resources: 60
-------------------------------------------------------
Generators_variability.csv Successfully Read!
Validating time basis
ELCC input

## Results